This is a jupyter notebook created for camera calibration practice.

In [41]:
import numpy as np
import cv2 as cv
import glob

import matplotlib.pyplot as plt

Step 1: Define parameters, setup settings, and read the images.

Reference:

- [OpenCV on Camera Calibration](https://docs.opencv.org/4.x/dc/dbb/tutorial_py_calibration.html);

In [42]:
# Define checkerboard_dims as (9, 6) 
checkboard_dims = (9, 6)

criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# Create objp as a zero array of shape (number of corners, 3), float32
# Set the first two columns of objp to the coordinate grid of corners
objp = np.zeros((7*6, 3), dtype=np.float32) # Refer to OpenCV on Camera Calibration.
objp[:, :2] = np.mgrid[0:7, 0:6].T.reshape(-1, 2)

# Initialize objpoints as an empty list
objpoints: list = []
# Initialize imgpoints as an empty list
imgpoints: list = []

# Load all checkerboard images using glob ('path/to/images/*.jpg')
images = glob.glob("calibration photos/*.jpg")

In [ ]:
# Test if the previous code segment is working.
images

Step 2: Find the locations of corners in the image, and store them in `imgpoints` and `objpoints`.

Reference:

- [OpenCV on Camera Calibration](https://docs.opencv.org/4.x/dc/dbb/tutorial_py_calibration.html);

In [44]:
# Initialize objpoints as an empty list
objpoints: list = []
# Initialize imgpoints as an empty list
imgpoints: list = []

# For each image in images:
for (image, i) in zip(images, range(len(images))):
    print(f"Processing image {i}")

    # Read the image
    img = cv.imread(images[i])
    # Convert the image to grayscale
    gray_img = cv.cvtColor(img, cv.COLOR_RGB2GRAY)

    # Find the chessboard corners in the grayscale image
    ret, corners = cv.findChessboardCorners(gray_img, (7, 6), None)
    # If corners are found:
    if ret == True:
        # Append objp to objpoints
        objpoints.append(objp)
        # Refine corner positions using cornerSubPix
        criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
        corners2 = cv.cornerSubPix(gray_img, corners, (11, 11), (-1, -1), criteria)
        # Append refined corners to imgpoints
        imgpoints.append(corners2)

        # Optionally, draw chessboard corners on the image
        # cv.drawChessboardCorners(img, (7, 6), corners2, ret)
        # Optionally, display the image with drawn corners
        # cv.imshow("Edited Image", img)
        # Wait for a short period
        # cv.waitKey(2000)

    print(f"Done Processing image {i}")
    
# Destroy all OpenCV windows
# cv.destroyAllWindows()

Processing image 0
Done Processing image 0
Processing image 1
Done Processing image 1
Processing image 2
Done Processing image 2
Processing image 3
Done Processing image 3
Processing image 4
Done Processing image 4
Processing image 5
Done Processing image 5
Processing image 6
Done Processing image 6
Processing image 7
Done Processing image 7
Processing image 8
Done Processing image 8
Processing image 9
Done Processing image 9
Processing image 10
Done Processing image 10
Processing image 11
Done Processing image 11
Processing image 12
Done Processing image 12
Processing image 13
Done Processing image 13
Processing image 14
Done Processing image 14
Processing image 15
Done Processing image 15
Processing image 16
Done Processing image 16
Processing image 17
Done Processing image 17
Processing image 18
Done Processing image 18
Processing image 19
Done Processing image 19


In [46]:
# plt.imshow(cv.imread(images[10]))
len(imgpoints)

3

Step 3: Calibrate the camera based on the obtained outcome, and verify the calibration.

Reference:

- [OpenCV on Camera Calibration](https://docs.opencv.org/4.x/dc/dbb/tutorial_py_calibration.html);

In [64]:
gray_img[::-1]

array([[ 76,  75,  74, ..., 103, 101,  95],
       [ 70,  69,  68, ..., 109, 107, 101],
       [ 74,  73,  72, ..., 109, 107, 103],
       ...,
       [153, 148, 146, ..., 131, 134, 136],
       [156, 153, 150, ..., 140, 143, 144],
       [153, 153, 152, ..., 143, 146, 146]], dtype=uint8)

In [68]:
# Calibrate the camera using calibrateCamera with objpoints, imgpoints, and image size
# Get the camera matrix, distortion coefficients, rotation vectors, and translation vectors
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray_img.shape[::-1], None, None)

# Save the calibration results (camera matrix, distortion coefficients) to a file.
np.savez_compressed("/home/nahida/Documents/GitHub/bwsi_laboratory_2024/week_1_Hw/camera_calibration_photo_mosaic/camera.npz",
        cam_mat = mtx, dist_coef = dist, rot_vec = rvecs, tra_vec = tvecs) 
# A common and convenient format for storing camera calibration data is the NumPy .npz file format,
    # which allows you to store multiple NumPy arrays in a single compressed file.


In [73]:
# Verify the calibration:
def verify_calibration(obj_points, img_points, r_vecs, t_vecs, matrix, distortion):
    # Initialize mean_error to 0
    mean_error = 0
    # For each pair of object points and image points:
    for objpoint, imgpoint, rvec, tvec in zip(obj_points, img_points, r_vecs, t_vecs):
        # Project the object points to image points using projectPoints
        imgpoints2, _ = cv.projectPoints(objpoint, rvec, tvec, matrix, distortion)
        # Compute the error between the projected and actual image points
        error = cv.norm(imgpoint, imgpoints2, cv.NORM_L2) / len(imgpoints2)
        # Accumulate the error
        mean_error += error

    # Compute the average error
    mean_error /= len(obj_points)
    # Print the total average error
    # print(mean_error)
    return mean_error


In [74]:
verify_calibration(objpoints, imgpoints, rvecs, tvecs, mtx, dist)

0.07728077072556332